In [1]:
import numpy as np #thư viện tính toán
import pandas as pd #thư viện xử lý dữ liệu
import matplotlib.pyplot as plt #thư viện vẽ đồ thị
import os #thư viện hệ thống
import cv2 #thư viện xử lý ảnh

In [2]:
df = pd.read_csv('labelimgcopy.csv') #đọc file csv chứa các nhãn
df.head() #hiển thị 5 dòng đầu tiên

,filepath,xmin,xmax,ymin,ymax
0,./imgcopy\Cars0.xml,226,419,125,173
1,./imgcopy\Cars1.xml,134,262,128,160
2,./imgcopy\Cars10.xml,140,303,5,148
3,./imgcopy\Cars100.xml,175,214,114,131
4,./imgcopy\Cars101.xml,167,240,202,220


In [3]:
import xml.etree.ElementTree as xet #thư viện xử lý file xml

In [4]:
filename = df['filepath'][4] #lấy tên file ảnh đầu tiên
filename #hiển thị tên file

'./imgcopy\\Cars101.xml'

In [5]:
#Hàm getFileName trả về tên file ảnh
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text #lấy tên file ảnh
    filepath_image = os.path.join('./imgcopy',filename_image) #tạo đường dẫn đến file ảnh
    return filepath_image #trả về tên file ảnh

In [6]:
getFilename(filename) #gọi hàm getFilename lấy tên file ảnh đầu tiên

'./imgcopy\\Cars101.png'

In [7]:
image_path = list(df['filepath'].apply(getFilename)) #lấy tên file ảnh từ tập dữ liệu , .apply() là hàm áp dụng hàm getFilename cho tất cả các dòng trong tập dữ liệu
image_path 

['./imgcopy\\Cars0.png',
 './imgcopy\\Cars1.png',
 './imgcopy\\Cars10.png',
 './imgcopy\\Cars100.png',
 './imgcopy\\Cars101.png',
 './imgcopy\\Cars102.png',
 './imgcopy\\Cars103.png',
 './imgcopy\\Cars104.png',
 './imgcopy\\Cars105.png',
 './imgcopy\\Cars106.png',
 './imgcopy\\Cars107.png',
 './imgcopy\\Cars108.png',
 './imgcopy\\Cars109.png',
 './imgcopy\\Cars11.png',
 './imgcopy\\Cars110.png',
 './imgcopy\\Cars111.png',
 './imgcopy\\Cars112.png',
 './imgcopy\\Cars113.png',
 './imgcopy\\Cars114.png',
 './imgcopy\\Cars115.png',
 './imgcopy\\Cars116.png',
 './imgcopy\\Cars117.png',
 './imgcopy\\Cars118.png',
 './imgcopy\\Cars119.png',
 './imgcopy\\Cars12.png',
 './imgcopy\\Cars120.png',
 './imgcopy\\Cars121.png',
 './imgcopy\\Cars122.png',
 './imgcopy\\Cars123.png',
 './imgcopy\\Cars124.png',
 './imgcopy\\Cars125.png',
 './imgcopy\\Cars126.png',
 './imgcopy\\Cars127.png',
 './imgcopy\\Cars128.png',
 './imgcopy\\Cars129.png',
 './imgcopy\\Cars13.png',
 './imgcopy\\Cars130.png',
 './imgco

#### verify image and output (hiển thị ảnh đầu vào và vẽ bounding box lên ảnh đó)

In [8]:
file_path = image_path[0]
file_path 

'./imgcopy\\Cars0.png'

In [9]:
img = cv2.imread(file_path) #đọc file ảnh

cv2.namedWindow('example',cv2.WINDOW_NORMAL) #tạo cửa sổ có tên là example
cv2.imshow('example',img) #hiển thị ảnh trong cửa sổ example
cv2.waitKey(0) #đợi người dùng nhấn phím bất kì
cv2.destroyAllWindows() #đóng cửa sổ

In [10]:
# 1093	1396	645	727
cv2.rectangle(img,(226,125),(419,173),(0,255,0),3) #vẽ hình chữ nhật bounding box trên ảnh, (0,255,0) là màu xanh lá cây, 3 là độ dày của đường viền, (1093,645) là tọa độ góc trái trên, (1396,727) là tọa độ góc phải dưới
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Data Preprocessing (tiền xử lý dữ liệu)

In [11]:
from sklearn.model_selection import train_test_split #thư viện chia tập dữ liệu thành tập train và tập test
from tensorflow.keras.preprocessing.image import load_img, img_to_array #thư viện xử lý ảnh, load_img() để đọc ảnh, img_to_array() để chuyển ảnh thành mảng numpy

In [12]:
labels = df.iloc[:,1:].values #lấy tất cả các cột từ cột thứ 2 trở đi (lấy xmin, xmax, ymin, ymax)

In [13]:
data = []  #khởi tạo mảng data lưu trữ ảnh xám đã được chuyển thành mảng numpy
output = [] #khởi tạo mảng output lưu trữ các nhãn (vị trí bounding box) đã điều chỉnh sau khi resize ảnh
for ind in range(len(image_path)): #duyệt qua tất cả các ảnh
    image = image_path[ind] #lấy tên file ảnh
    img_arr = cv2.imread(image) #đọc ảnh
    h,w,d = img_arr.shape #lấy chiều cao, chiều rộng, số kênh màu của ảnh
    # prepprocesing tiền xử lý ảnh
    load_image = load_img(image,target_size=(224,224)) #đọc ảnh, resize ảnh về kích thước 224x224 (vì mô hình YOLOv3 chỉ nhận ảnh có kích thước 224x224)
    load_image_arr = img_to_array(load_image) #chuyển ảnh thành mảng numpy
    norm_load_image_arr = load_image_arr/255.0 # normalization ảnh, đưa các giá trị trong mảng về khoảng 0-1
    # normalization to labels, tính toán lại các giá trị của bounding box theo tỉ lệ của ảnh
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w #tính toán lại tọa độ bounding box sau khi resize ảnh
    nymin,nymax = ymin/h,ymax/h #tính toán lại tọa độ bounding box sau khi resize ảnh
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output, tọa độ bounding box sau khi resize ảnh
    # -------------- append
    data.append(norm_load_image_arr) #thêm ảnh đã được xử lý thành ảnh xám vào mảng data
    output.append(label_norm) #thêm tọa độ bounding box đã được xử lý vào mảng output

In [14]:
X = np.array(data,dtype=np.float32) #chuyển mảng data thành mảng numpy
y = np.array(output,dtype=np.float32) #chuyển mảng output thành mảng numpy

In [15]:
X.shape,y.shape #hình dạng của mảng X và y, X có 225 ảnh, mỗi ảnh có kích thước 224x224x3, y có 225 ảnh, mỗi ảnh có 4 giá trị tọa độ bounding box

((658, 224, 224, 3), (658, 4))

In [16]:
#chia tập dữ liệu thành tập train và tập test
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape #hình dạng của tập train và tập test sau khi chia, tập train có 180 ảnh, tập test có 45 ảnh

((526, 224, 224, 3), (132, 224, 224, 3), (526, 4), (132, 4))

### Deep Learning Model (Mô hình học sâu)

In [1]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2 #import các mô hình MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input #import các lớp Dense, Dropout, Flatten, Input để xây dựng mô hình
from tensorflow.keras.models import Model #import Model để xây dựng mô hình
import tensorflow as tf #import tensorflow

In [2]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3))) #khởi tạo mô hình InceptionResNetV2, weights="imagenet" để load các trọng số của mô hình đã được huấn luyện trên ImageNet, include_top=False để không sử dụng lớp output của mô hình, input_tensor=Input(shape=(224,224,3)) để chỉ định kích thước của ảnh đầu vào
inception_resnet.trainable=False #đóng băng các trọng số của mô hình InceptionResNetV2
# ---------------------
headmodel = inception_resnet.output #cấu hình output của mô hình InceptionResNetV2
headmodel = Flatten()(headmodel) #thêm lớp Flatten để chuyển đổi các giá trị của mảng 2 chiều thành mảng 1 chiều, có tác dụng giảm số chiều của mảng giúp giảm số lượng tham số trong mô hình, lớp này có nhiệm vụ duỗi ma trận 2 chiều thành một vector để làm đầu vào cho lớp fully connected tiếp theo
headmodel = Dense(500,activation="relu")(headmodel) #thêm lớp Dense với 500 nút ẩn và hàm kích hoạt relu, lớp Dense là lớp fully connected, có tác dụng tạo ra các đặc trưng từ các đặc trưng ở các lớp trước đó để đưa vào lớp output để dự đoán kết quả, 500 tương ứng với số lượng đặc trưng được tạo ra, dùng hàm kích hoạt relu để giảm thiểu hiện tượng overfitting
headmodel = Dense(250,activation="relu")(headmodel) #thêm lớp Dense với 250 nút ẩn và hàm kích hoạt relu, 250 tương ứng với số lượng đặc trưng được tạo ra, lúc nãy là 500 giờ là 250, số lượng đặc trưng được tạo ra sẽ giảm dần theo thời gian vì các đặc trưng càng cao thì càng chứa nhiều thông tin, nên cần giảm số lượng đặc trưng xuống để tránh overfitting
headmodel = Dense(4,activation='sigmoid')(headmodel) #thêm lớp Dense với 4 nút ẩn và hàm kích hoạt sigmoid, lớp Dense cuối cùng sẽ dự đoán kết quả, 4 nút ẩn tương ứng với 4 giá trị tọa độ bounding box, dùng hàm kích hoạt sigmoid để đưa ra kết quả dự đoán là giá trị nằm trong khoảng [0,1]
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel) #xây dựng mô hình bằng cách kết nối input của mô hình InceptionResNetV2 với output của headmodel

In [3]:
# complie model: biên dịch mô hình
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics = ["accuracy", tf.metrics.AUC()]) #sử dụng hàm mất mát MSE, sử dụng thuật toán tối ưu Adam với learning_rate=1e-4, sử dụng độ đo accuracy và AUC để đánh giá mô hình
model.summary() #xem tóm tắt mô hình
#Đếm xem mô hình có bao nhiêu lớp
len(model.layers)
#Đếm xem mô hình có bao nhiêu tham số
model.count_params()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 batch_normalization_5 (BatchNo  (None, 25, 25, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 25, 25, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_10 (BatchN  (None, 25, 25, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 25, 25, 64)  192         ['conv2d_11[0][0]']              
 ormalizat

 activation_21 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 25, 25, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 25, 25, 48)   13824       ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_19 (BatchN  (None, 25, 25, 32)  96          ['conv2d_19[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_22 (BatchN  (None, 25, 25, 48)  144         ['conv2d_22[0][0]']              
 ormalizat

                                                                                                  
 activation_24 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 25, 25, 64)   0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 25, 25, 128)  0           ['activation_24[0][0]',          
                                                                  'activation_26[0][0]',          
                                                                  'activation_29[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 25, 25, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_37 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_37[0][0]'] 
                                                                                                  
 activation_40 (Activation)     (None, 25, 25, 48)   0           ['batch_normalization_40[0][0]'] 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 25, 25, 32)   10240       ['block35_4_ac[0][0]']           
          

...

 batch_normalization_175 (Batch  (None, 5, 5, 192)   576         ['conv2d_175[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_178 (Batch  (None, 5, 5, 256)   768         ['conv2d_178[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_175 (Activation)    (None, 5, 5, 192)    0           ['batch_normalization_175[0][0]']
                                                                                                  
 activation_178 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_178[0][0]']
                                                                                                  
 block8_4_

                                                                                                  
 activation_186 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_186[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 5, 5, 448)    0           ['activation_183[0][0]',         
                                                                  'activation_186[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 5, 5, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 block8_6 (Lambda)              (None, 5, 5, 2080)   0           ['block8_5_ac[0][0]',            
                                                                  'block8_6_conv[0][0]']          
          

                                                                                                  
 block8_8 (Lambda)              (None, 5, 5, 2080)   0           ['block8_7_ac[0][0]',            
                                                                  'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 5, 5, 2080)   0           ['block8_8[0][0]']               
                                                                                                  
 conv2d_196 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_196 (Batch  (None, 5, 5, 192)   576         ['conv2d_196[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 5, 5, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 flatten (Flatten)              (None, 38400)        0           ['conv_7b_ac[0][0]']             
                                                                                                  
 dense (Dense)                  (None, 500)          19200500    ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 250)          125250      ['dense[0][0]']                  
                                                                                                  
 dense_2 (

73663490

In [4]:
#import keras đề dùng keras.utils.plot_model để vẽ mô hình
import keras

keras.utils.plot_model(model, show_shapes=True)

### model training

In [20]:
from tensorflow.keras.callbacks import TensorBoard #import TensorBoard để lưu lại quá trình huấn luyện mô hình

In [21]:
tfb = TensorBoard('object_detection') #khởi tạo TensorBoard, 'object_detection' là tên thư mục lưu lại quá trình huấn luyện mô hình

In [ ]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb]) #tiến hành huấn luyện mô hình, batch_size=10 là số lượng ảnh trong mỗi batch (tùy thuộc vào tốc độ máy tính), epochs=100 là số lần lặp lại quá trình huấn luyện, validation_data=(x_test,y_test) để kiểm tra độ chính xác của mô hình trên tập test sau mỗi lần lặp lại quá trình huấn luyện, callbacks=[tfb] để lưu lại quá trình huấn luyện mô hình

Epoch 1/100
53/53 [==============================] - 56s 908ms/step - loss: 0.1434 - accuracy: 0.4582 - auc: 0.8749 - val_loss: 0.1252 - val_accuracy: 0.4621 - val_auc: 0.0000e+00
Epoch 2/100
53/53 [==============================] - 46s 868ms/step - loss: 0.1273 - accuracy: 0.4525 - auc: 0.8749 - val_loss: 0.1274 - val_accuracy: 0.4621 - val_auc: 0.0000e+00
Epoch 3/100
53/53 [==============================] - 45s 861ms/step - loss: 0.1238 - accuracy: 0.4525 - auc: 0.8752 - val_loss: 0.1226 - val_accuracy: 0.4621 - val_auc: 0.0000e+00
Epoch 4/100
53/53 [==============================] - 47s 882ms/step - loss: 0.1231 - accuracy: 0.4525 - auc: 0.8752 - val_loss: 0.1166 - val_accuracy: 0.4621 - val_auc: 0.0000e+00
Epoch 5/100
53/53 [==============================] - 48s 913ms/step - loss: 0.1199 - accuracy: 0.4525 - auc: 0.8752 - val_loss: 0.1184 - val_accuracy: 0.4621 - val_auc: 0.0000e+00
Epoch 6/100
53/53 [==============================] - 50s 944ms/step - loss: 0.1189 - accuracy: 0.452

In [23]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=100) #tiến hành huấn luyện mô hình, batch_sise là trọng số cập nhật sau mỗi bó (tùy thuộc vào tốc độ máy tính), epochs=100 là số lần lặp lại quá trình huấn luyện, validation_data=(x_test,y_test) để kiểm tra độ chính xác của mô hình trên tập test sau mỗi lần lặp lại quá trình huấn luyện, callbacks=[tfb] để lưu lại quá trình huấn luyện mô hình, batch_size là trọng số được cập nhật sau mỗi bó, chọn batch_size nhỏ sẽ giúp mô hình hội tụ nhanh hơn, nhưng nếu batch_size quá nhỏ thì sẽ làm giảm độ chính xác của mô hình, chọn epochs quá lớn sẽ làm mô hình học quá mức, chọn epochs quá nhỏ sẽ làm mô hình chưa học đủ, chọn epochs thích hợp sẽ làm mô hình học đủ và hội tụ nhanh hơn

Epoch 101/200
53/53 [==============================] - 50s 939ms/step - loss: 8.6216e-04 - accuracy: 0.9563 - auc: 0.9993 - val_loss: 0.0101 - val_accuracy: 0.8333 - val_auc: 0.0000e+00
Epoch 102/200
53/53 [==============================] - 50s 943ms/step - loss: 7.9003e-04 - accuracy: 0.9487 - auc: 1.0000 - val_loss: 0.0099 - val_accuracy: 0.8333 - val_auc: 0.0000e+00
Epoch 103/200
53/53 [==============================] - 49s 930ms/step - loss: 8.8721e-04 - accuracy: 0.9449 - auc: 0.9993 - val_loss: 0.0110 - val_accuracy: 0.8333 - val_auc: 0.0000e+00
Epoch 104/200
53/53 [==============================] - 51s 958ms/step - loss: 9.5866e-04 - accuracy: 0.9563 - auc: 0.9986 - val_loss: 0.0105 - val_accuracy: 0.8258 - val_auc: 0.0000e+00
Epoch 105/200
53/53 [==============================] - 51s 975ms/step - loss: 9.0134e-04 - accuracy: 0.9430 - auc: 0.9995 - val_loss: 0.0101 - val_accuracy: 0.8258 - val_auc: 0.0000e+00
Epoch 106/200
53/53 [==============================] - 51s 959ms/step 

53/53 [==============================] - 48s 911ms/step - loss: 0.0011 - accuracy: 0.9544 - auc: 0.9995 - val_loss: 0.0108 - val_accuracy: 0.8561 - val_auc: 0.0000e+00
Epoch 190/200
53/53 [==============================] - 51s 972ms/step - loss: 9.2303e-04 - accuracy: 0.9582 - auc: 0.9993 - val_loss: 0.0106 - val_accuracy: 0.8409 - val_auc: 0.0000e+00
Epoch 191/200
53/53 [==============================] - 50s 953ms/step - loss: 7.9704e-04 - accuracy: 0.9715 - auc: 0.9998 - val_loss: 0.0101 - val_accuracy: 0.8409 - val_auc: 0.0000e+00
Epoch 192/200
53/53 [==============================] - 51s 974ms/step - loss: 7.4627e-04 - accuracy: 0.9506 - auc: 0.9995 - val_loss: 0.0105 - val_accuracy: 0.8561 - val_auc: 0.0000e+00
Epoch 193/200
53/53 [==============================] - 50s 946ms/step - loss: 7.1414e-04 - accuracy: 0.9639 - auc: 0.9976 - val_loss: 0.0101 - val_accuracy: 0.8561 - val_auc: 0.0000e+00
Epoch 194/200
53/53 [==============================] - 49s 934ms/step - loss: 9.8060e-04

In [24]:
model.save('./models/object_detection_2_renew.h5') #lưu lại mô hình